In [41]:
from utils.utils import read_mongo
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [229]:
df = read_mongo("changeorg", "petitions_scraped", {"id": {"$gt": 0}})

In [242]:
    list_targets = np.asarray(df["targets"].apply(
        lambda x: [[target["display_name"],
                    target["additional_data"]["title"],
                    target["additional_data"]["state"],
                    target["id"],
                    target["description"],
                    ]
                   for target in x if target["type"] == "Politician"]))

In [243]:
flat_target_list = []
for targets in list_targets:
    for target in targets:
        flat_target_list.append(target)
list_df = pd.DataFrame(flat_target_list)

        

In [244]:
list_df["slug"] = list_df[0].apply(lambda x: x.replace(" ", "_"))

In [245]:
list_df = list_df.drop_duplicates()

In [246]:
list_df.shape

(6710, 6)

In [157]:
def getParty(slug):
    url = "https://ballotpedia.org/"
    html = requests.get(url + slug).content
    soup = BeautifulSoup(html, "lxml")
    d = soup.findAll("a", href=True, text='Democratic')
    r = soup.findAll("a", href=True, text='Republican')
    if len(d) > len(r):
        return "D"
    elif len(r) > len(d):
        return "R"
    else:
        return None

In [207]:
list_df.head()

,0,1,2,3,4,slug
0,Dannel Malloy,Governor,CT,763329,None,Dannel_Malloy
1,Eleanor Norton,Representative,DC,750198,,Eleanor_Norton
2,Barack Obama,President,None,750542,Barack Obama is the 44th and current President...,Barack_Obama
3,Cory Booker,Senator,NJ,750574,,Cory_Booker
4,Robert Menendez,Senator,NJ,750575,Robert Menendez is the senior United States Se...,Robert_Menendez


In [226]:
list_df["party"]=list_df.apply(lambda x: "D" if x[4] is not None and x[4].lower().find("democrat")>=0 else None, axis =1)

list_df["party"] = list_df.apply(lambda x: "R" if x[4] is not None and x[4].lower().find("republican")>=0 else x["party"], axis =1)

In [227]:
list_df

,0,1,2,3,4,slug,party
0,Dannel Malloy,Governor,CT,763329,None,Dannel_Malloy,None
1,Eleanor Norton,Representative,DC,750198,,Eleanor_Norton,None
2,Barack Obama,President,None,750542,Barack Obama is the 44th and current President...,Barack_Obama,None
3,Cory Booker,Senator,NJ,750574,,Cory_Booker,None
4,Robert Menendez,Senator,NJ,750575,Robert Menendez is the senior United States Se...,Robert_Menendez,D
5,Donald Norcross,Representative,NJ,2404161,None,Donald_Norcross,None
6,Bill Pascrell,Representative,NJ,750355,None,Bill_Pascrell,None
7,Donald Payne,Representative,NJ,750356,None,Donald_Payne,None
8,Bonnie Watson Coleman,Representative,NJ,2404171,None,Bonnie_Watson_Coleman,None
9,Albio Sires,Representative,NJ,750354,None,Albio_Sires,None


In [163]:
sample["Party"] = sample.apply(lambda x: x["slug"] if x["Party"] is None else x, axis =1) 

/Users/natalie/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [241]:
type(int(sample[:1]["id"][0]))

int

In [111]:
sample["Party"]= sample.apply(lambda x: getParty(x))

/Users/natalie/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [166]:
sample.columns = ["id", "description", "name", "position", "state", "slug", "party"]

In [235]:
list_df[5:10][:1]

,0,1,2,3,4,slug,party
5,Donald Norcross,Representative,NJ,2404161,None,Donald_Norcross,None


In [ ]:
    #Get MongoDB
    mc = MongoClient()
    db = mc['changeorg']
    db.drop_collection("congress")
    mongo_congress = db["congress"]